In [1]:
import sys
sys.path.append(
    '/home/macrodrigues/code/macrodrigues/my_projects/butterfly_detector/')
from scripts.transfer_learning import PATH, PATH_TEST
from scripts.transfer_learning import imgs_gen, train_gen, val_gen
from scripts.transfer_learning import compile_model, train_model 
from scripts.transfer_learning import plot_history, check_accuracy


2022-02-02 20:55:49.615747: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-02-02 20:55:49.616046: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Found 2808 images belonging to 12 classes.
Found 838 images belonging to 12 classes.


In [8]:
model_test = compile_model()

In [8]:
history = train_model(model_test, name_to_save='model_test')


In [3]:
check_accuracy(pick_model='model')


TITLE: Unknown
PREDICTIONS: {'Unknown': 1.0}
BEST PREDICTION: Unknown
RESULT Good prediction!


TITLE: Colias Croceus
PREDICTIONS: {'Colias Croceus': 1.0}
BEST PREDICTION: Colias Croceus
RESULT Good prediction!


TITLE: Unknown
PREDICTIONS: {'Unknown': 1.0}
BEST PREDICTION: Unknown
RESULT Good prediction!


TITLE: Maniola Jurtina
PREDICTIONS: {'Coenonympha Pamphilus': 5.2121176e-14, 'Maniola Jurtina': 1.0}
BEST PREDICTION: Maniola Jurtina
RESULT Good prediction!


TITLE: Iphiclides Feisthamelii
PREDICTIONS: {'Iphiclides Feisthamelii': 1.0}
BEST PREDICTION: Iphiclides Feisthamelii
RESULT Good prediction!


TITLE: Aricia Cramera
PREDICTIONS: {'Aricia Cramera': 1.0}
BEST PREDICTION: Aricia Cramera
RESULT Good prediction!


TITLE: Unknown
PREDICTIONS: {'Unknown': 1.0}
BEST PREDICTION: Unknown
RESULT Good prediction!


TITLE: Coenonympha Pamphilus
PREDICTIONS: {'Coenonympha Pamphilus': 1.0}
BEST PREDICTION: Coenonympha Pamphilus
RESULT Good prediction!


TITLE: Maniola Jurtina
PREDICTIONS: 